In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_rop',
    'order_report',
    proc_date,
    'temp_rop_order_report'
)

In [0]:
spark.sql(f""" CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_rop.order_report (
  order_id STRING,
  region STRING,
  city STRING,
  vendor_id STRING,
  vendor_name STRING,
  vendor_address STRING,
  sub_range_id STRING,
  sub_range_name STRING,
  store_id STRING,
  store_name STRING,
  store_address STRING,
  order_user STRING,
  product_stt STRING,
  product_id STRING,
  product_name STRING,
  barcode STRING,
  baseuom STRING,
  price STRING,
  purchase_order_quantity STRING,
  purchase_order_quantity_delivery STRING,
  purchase_order_quantity_delivered_actual STRING,
  order_date STRING,
  order_date_request STRING,
  order_date_expected_delivery STRING,
  time_frame_of_delivery STRING,
  delivery_confirmation_date STRING,
  time_frame_for_delivery_confirmation STRING,
  delivery_date STRING,
  status STRING,
  upd_date STRING,
  note_1 STRING,
  note_2 STRING,
  proc_date DATE,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_rop.order_report
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""INSERT INTO {catalog_name}.udp_wcm_silver_rop.order_report 
          SELECT order_id,
                region,
                city,
                vendor_id,
                vendor_name,
                vendor_address,
                sub_range_id,
                sub_range_name,
                store_id,
                store_name,
                store_address,
                order_user,
                product_stt,
                product_id,
                product_name,
                barcode,
                baseuom,
                price,
                purchase_order_quantity,
                purchase_order_quantity_delivery,
                purchase_order_quantity_delivered_actual,
                TO_DATE(SUBSTRING(order_date, 1, INSTR(order_date, ' ') - 1), 'M/d/yyyy') AS order_date,
                TO_DATE(SUBSTRING(order_date_request, 1, INSTR(order_date_request, ' ') - 1), 'M/d/yyyy') AS order_date_request,
                CASE
                  WHEN order_date_expected_delivery LIKE '%AM%' 
                       OR order_date_expected_delivery LIKE '% 12:%'
                       THEN TO_TIMESTAMP(order_date_expected_delivery, 'M/d/yyyy h:m:s a')

                  WHEN order_date_expected_delivery LIKE '%PM%' 
                       AND order_date_expected_delivery NOT LIKE '% 12:%'
                       THEN TIMESTAMPADD(
                              HOUR, 
                              12, 
                              TO_TIMESTAMP(order_date_expected_delivery, 'M/d/yyyy h:m:s a')
                            )
                  ELSE NULL
                END AS order_date_expected_delivery,
                time_frame_of_delivery,
                CASE
                  WHEN delivery_confirmation_date LIKE '%AM%' 
                       OR delivery_confirmation_date LIKE '% 12:%' 
                       THEN TO_TIMESTAMP(delivery_confirmation_date, 'M/d/yyyy h:m:s a')

                  WHEN delivery_confirmation_date LIKE '%PM%' 
                       AND delivery_confirmation_date NOT LIKE '% 12:%' 
                       THEN TIMESTAMPADD(
                              HOUR, 
                              12, 
                              TO_TIMESTAMP(delivery_confirmation_date, 'M/d/yyyy h:m:s a')
                            )
                  ELSE NULL
                END AS delivery_confirmation_date,
                time_frame_for_delivery_confirmation,
                CASE
                  WHEN delivery_date LIKE '%AM%' 
                       OR delivery_date LIKE '% 12:%'
                       THEN TO_TIMESTAMP(delivery_date, 'M/d/yyyy h:m:s a')

                  WHEN delivery_date LIKE '%PM%' 
                       AND delivery_date NOT LIKE '% 12:%'
                       THEN TIMESTAMPADD(
                              HOUR,
                              12,
                              TO_TIMESTAMP(delivery_date, 'M/d/yyyy h:m:s a')
                            )
                  ELSE NULL
                END AS delivery_date,
                status,
                CASE
                  WHEN upd_date LIKE '%AM%' 
                       OR upd_date LIKE '% 12:%' 
                       THEN TO_TIMESTAMP(upd_date, 'M/d/yyyy h:m:s a')

                  WHEN upd_date LIKE '%PM%' 
                       AND upd_date NOT LIKE '% 12:%' 
                       THEN TIMESTAMPADD(
                              HOUR, 
                              12, 
                              TO_TIMESTAMP(upd_date, 'M/d/yyyy h:m:s a')
                            )
                  ELSE NULL
                END AS upd_date,
                note_1,
                note_2,
                proc_date,
                file_creation_ts FROM temp_rop_order_report
""")